In [1]:
%set_env PROJECT_PATH=/Users/gclyne/thesis

#generate sliding window timeseries of 30 years for each grid cell? 
import pandas as pd
import torch as T

import numpy as np
from other.utils import readCoordinates
cesm_data = pd.read_csv('/Users/gclyne/thesis/data/cesm_data.csv')
managed_forest_coordinates = readCoordinates('managed_coordinates.csv',is_grid_file=False)
from other.constants import MODEL_INPUT_VARIABLES,MODEL_TARGET_VARIABLES,RAW_CMIP_VARIABLES
from time_series_transformer import CMIPTimeSeriesDataset,TimeSeriesTransformer
inputs = MODEL_INPUT_VARIABLES + MODEL_TARGET_VARIABLES + ['# year','lat','lon']
cesm_data = cesm_data[inputs]
ds = CMIPTimeSeriesDataset(cesm_data,managed_forest_coordinates)


env: PROJECT_PATH=/Users/gclyne/thesis


/Users/gclyne/thesis/env/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
from time_series_transformer import generate_square_subsequent_mask
num_of_epochs = 3
#split train and validation
test_validation_batch_size=100
learning_rate = 0.001
train_set_size = int(len(ds) * 0.75)
valid_set_size = int(len(ds) * 0.2)
hold_out = int(len(ds) * .05) + 1
train,validation,hold_out = T.utils.data.random_split(ds, [train_set_size, valid_set_size,hold_out], generator=T.Generator().manual_seed(0))
# train.loc[:,input_variable_tuple] = min_max_scaler.fit_transform(train.loc[:,input_variable_tuple])
# validation.loc[:,input_variable_tuple] = min_max_scaler.fit_transform(validation.loc[:,input_variable_tuple])
train = T.utils.data.Subset(train,range(0,1000))
validation = T.utils.data.Subset(validation,range(0,1000))
model = TimeSeriesTransformer(input_size=len(MODEL_INPUT_VARIABLES),dec_seq_len=30, batch_first=True,
dim_val=24,n_encoder_layers=64,n_decoder_layers=64,n_heads=8,num_predicted_features=len(MODEL_TARGET_VARIABLES))
loss_function = T.nn.MSELoss()
train_ldr = T.utils.data.DataLoader(train,batch_size=test_validation_batch_size,shuffle=True)
validation_ldr = T.utils.data.DataLoader(validation,batch_size=test_validation_batch_size,shuffle=True)
hold_out_loader = T.utils.data.DataLoader(hold_out,batch_size=10,shuffle=True)

optimizer = T.optim.Adam(model.parameters(), lr=learning_rate)
training_losses = []
validation_losses = []
min_valid_loss = np.inf

# Input length
enc_seq_len = 30
# Output length
output_sequence_length = 5
# Make src mask for decoder with size:
tgt_mask = generate_square_subsequent_mask(
    dim1=output_sequence_length,
    dim2=output_sequence_length
   )

src_mask = generate_square_subsequent_mask(
    dim1=output_sequence_length,
    dim2=enc_seq_len
    )


In [16]:
for epoch in range(num_of_epochs):
    train_loss = 0
    model.train()
    for src,tgt,tgt_y in train_ldr:
        optimizer.zero_grad() #clears old gradients from previous steps 
        pred_tgt = model(src.float(),tgt.float())
        loss = loss_function(pred_tgt, tgt_y.float())
        print(pred_tgt[0])
        print(tgt_y.float()[0])
        loss.backward() #compute gradient
        optimizer.step() #take step based on gradient
        train_loss += loss.item()
        print(src.size())
        # print(loss)
        break

    # model.eval()
    # valid_loss = 0
    # for src,tgt,tgt_y in validation_ldr:
    #     # print(X.data.numpy(),y.data.numpy())
    #     pred_y = model(src.float(),tgt.float())
    #     # print(pred_y.data.numpy())
    #     test_loss = loss_function(pred_y,tgt_y.float())
    #     valid_loss += test_loss.item()
    # print(f'Epoch {epoch+1} \t\t Training Loss: {train_loss} \t\t Validation Loss: {valid_loss}')
    # if min_valid_loss > valid_loss:
    #     print(f'Validation Loss Decreased({min_valid_loss:.6f}--->{valid_loss:.6f})')
    #     min_valid_loss = valid_loss



tensor([[3.8694, 1.5139, 1.2082, 0.6789, 0.3654, 0.3640, 2.1380, 0.6519],
        [3.8001, 1.1534, 1.2091, 0.7746, 0.2024, 0.2166, 2.0948, 0.3160],
        [3.8231, 1.2879, 1.4235, 0.5612, 0.3095, 0.4227, 2.3258, 0.6780],
        [3.9493, 1.3382, 1.0924, 0.8527, 0.1668, 0.6911, 2.0875, 0.5391],
        [3.7857, 1.5536, 1.3337, 0.5815, 0.3276, 0.3321, 2.3052, 0.5029]],
       grad_fn=<SelectBackward0>)
tensor([[28.8071,  3.1152,  7.6414,  0.5379,  0.1584,  1.9325,  5.3650,  0.2770],
        [28.7942,  3.1199,  7.7563,  0.5759,  0.1639,  1.9058,  5.4675,  0.2692],
        [28.8128,  3.1264,  7.5698,  0.5856,  0.1495,  1.9184,  5.3614,  0.2772],
        [28.7965,  3.1011,  7.6460,  0.5579,  0.1562,  1.9414,  5.3884,  0.2697],
        [28.7920,  3.1484,  7.8156,  0.5924,  0.1654,  1.9042,  5.4965,  0.2499]])
<built-in method size of Tensor object at 0x131c876d0>
tensor([[3.8717, 1.2725, 1.3050, 0.9441, 0.0514, 0.7968, 1.8555, 0.5588],
        [3.9678, 1.0969, 1.3079, 0.7807, 0.1347, 0.1205

In [9]:

hold_out_pred_list = []
hold_out_target = []

for src,tgt,tgt_y in hold_out_loader:
    final_pred = model(src.float(),tgt.float())
    loss = loss_function(final_pred,tgt_y.float())
    hold_out_pred_list.append(final_pred.detach().numpy()[0])
    hold_out_target.append(tgt_y.detach().numpy()[0])

In [10]:
hold_out_pred_list

[array([[3.66596   , 1.1404965 , 1.0850669 , 0.7515271 , 0.40598235,
         0.3456606 , 1.898637  , 0.7577963 ],
        [3.66596   , 1.1404965 , 1.0850668 , 0.75152713, 0.40598232,
         0.3456606 , 1.8986372 , 0.7577963 ],
        [3.66596   , 1.1404965 , 1.0850668 , 0.751527  , 0.40598235,
         0.34566057, 1.8986369 , 0.75779617],
        [3.66596   , 1.1404965 , 1.0850669 , 0.751527  , 0.40598235,
         0.34566063, 1.8986372 , 0.7577964 ],
        [3.6659596 , 1.1404965 , 1.0850668 , 0.751527  , 0.40598232,
         0.3456606 , 1.8986369 , 0.7577963 ]], dtype=float32),
 array([[3.6659596 , 1.1404965 , 1.0850667 , 0.751527  , 0.40598235,
         0.34566063, 1.8986369 , 0.75779617],
        [3.6659598 , 1.1404965 , 1.0850667 , 0.75152713, 0.40598232,
         0.3456606 , 1.898637  , 0.75779617],
        [3.66596   , 1.1404965 , 1.0850669 , 0.75152713, 0.40598232,
         0.3456606 , 1.8986372 , 0.75779617],
        [3.6659596 , 1.1404965 , 1.0850668 , 0.7515271 , 0.4059

In [11]:
hold_out_target

[array([[23.88440132,  4.9399848 ,  8.86634827,  1.36614144,  0.2423709 ,
          2.6383605 ,  6.00073862,  0.14895552],
        [23.92407227,  5.14823771,  9.29053402,  1.53403175,  0.23730327,
          2.53302336,  6.38779497,  0.16368692],
        [23.95458984,  5.27918768,  9.60387707,  1.44692373,  0.25005507,
          2.64857364,  6.60374117,  0.16537373],
        [23.9054184 ,  5.03631496,  9.01216888,  1.4375304 ,  0.23539527,
          2.63074732,  6.13904333,  0.14329386],
        [23.97964668,  5.38601446,  9.05493641,  1.60990989,  0.19399273,
          2.54905319,  6.31977034,  0.14452891]]),
 array([[23.60817146,  5.4657712 , 11.23816967,  1.92698753,  0.24458845,
          2.85995483,  7.99769211,  0.17172609],
        [23.61279869,  5.37968874, 11.44794369,  1.93230534,  0.24748065,
          2.88572216,  8.15876102,  0.17127091],
        [23.6383419 ,  5.32140064, 11.69495106,  1.86993742,  0.26034218,
          2.91917109,  8.32183456,  0.20215522],
        [23.60